## Import Libraries

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import DataLoader, TensorDataset, random_split

## Data Preparation

In [2]:
df=pd.read_excel("AirQualityUCI.xlsx")
df.dropna(inplace=True)
df['month'] = df['Date'].dt.month
df

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH,month
0,2004-03-10,18:00:00,2.6,1360.00,150,11.881723,1045.50,166.0,1056.25,113.0,1692.00,1267.50,13.600,48.875001,0.757754,3
1,2004-03-10,19:00:00,2.0,1292.25,112,9.397165,954.75,103.0,1173.75,92.0,1558.75,972.25,13.300,47.700000,0.725487,3
2,2004-03-10,20:00:00,2.2,1402.00,88,8.997817,939.25,131.0,1140.00,114.0,1554.50,1074.00,11.900,53.975000,0.750239,3
3,2004-03-10,21:00:00,2.2,1375.50,80,9.228796,948.25,172.0,1092.00,122.0,1583.75,1203.25,11.000,60.000000,0.786713,3
4,2004-03-10,22:00:00,1.6,1272.25,51,6.518224,835.50,131.0,1205.00,116.0,1490.00,1110.00,11.150,59.575001,0.788794,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9352,2005-04-04,10:00:00,3.1,1314.25,-200,13.529605,1101.25,471.7,538.50,189.8,1374.25,1728.50,21.850,29.250000,0.756824,4
9353,2005-04-04,11:00:00,2.4,1162.50,-200,11.355157,1027.00,353.3,603.75,179.2,1263.50,1269.00,24.325,23.725000,0.711864,4
9354,2005-04-04,12:00:00,2.4,1142.00,-200,12.374538,1062.50,293.0,603.25,174.7,1240.75,1092.00,26.900,18.350000,0.640649,4
9355,2005-04-04,13:00:00,2.1,1002.50,-200,9.547187,960.50,234.5,701.50,155.7,1041.00,769.75,28.325,13.550000,0.513866,4


In [4]:
for i in df.columns:
    if df[i].dtypes=="object":
        df[i] = df[i].astype('category')
        df[i] =df[i].cat.codes

In [7]:
targets = df[["PT08.S1(CO)"]].to_numpy()
inputs = df.drop(["PT08.S1(CO)","Date"], axis=1).to_numpy()  

In [8]:
inputs = torch.from_numpy(inputs)
inputs = inputs.float()
targets = torch.from_numpy(targets)
targets = targets.float()

In [9]:
inputs_mean = torch.mean(inputs, axis=0)
targets_mean = torch.mean(targets, axis=0)
inputs = inputs / inputs_mean  
targets = targets / targets_mean 

In [10]:
dataset = TensorDataset(inputs, targets)
a=len(dataset)
b=round(0.9*a)
c=a-b
train_dataset, val_dataset = torch.utils.data.dataset.random_split(dataset,[b,c])

In [11]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader=DataLoader(val_dataset, batch_size=64, shuffle=True)

## Define Two Models

In [12]:
model1 = nn.Sequential(nn.Linear(14, 8),
                      nn.Sigmoid(),
                       
                      nn.Linear(8, 4),
                      nn.Softmax(),
                       
                       
                      nn.Linear(4, 2),
                      nn.ReLU(),

                      
                      nn.Linear(2, 1),
                      nn.Sigmoid())

model2 = nn.Sequential(nn.Linear(14, 1),
                       nn.ReLU())

In [13]:
# Utility function to train the model
def fit(num_epochs, model, criterion, optimizer, train_loader):
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        # Train with batches of data
        for xb,yb in train_loader:
            # 1. Generate predictions
            pred = model(xb)
            # 2. Calculate loss
            loss = criterion(pred, yb)
            # 3. Compute gradients
            loss.backward()
            # 4. Update parameters using gradients
            optimizer.step()
            # 5. Reset the gradients to zero
            optimizer.zero_grad()
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

## Fit Model1

In [13]:
criterion = F.mse_loss
optimizer = torch.optim.SGD(model1.parameters(), lr=1e-4)
fit(100, model1, criterion, optimizer, train_loader)

Epoch [10/100], Loss: 0.2482
Epoch [20/100], Loss: 0.2078
Epoch [30/100], Loss: 0.1655
Epoch [40/100], Loss: 0.1415
Epoch [50/100], Loss: 0.2711
Epoch [60/100], Loss: 0.1984
Epoch [70/100], Loss: 0.1362
Epoch [80/100], Loss: 0.1757
Epoch [90/100], Loss: 0.1616
Epoch [100/100], Loss: 0.1993


## Fit Model1 with Regularization

In [14]:
criterion = F.mse_loss
optimizer = torch.optim.SGD(model2.parameters(),weight_decay=2, lr=1e-4)
fit(100, model1, criterion, optimizer, train_loader)

C:\Users\user\Anaconda3\lib\site-packages\torch\nn\modules\container.py:117: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Epoch [10/100], Loss: 0.2732
Epoch [20/100], Loss: 0.3728
Epoch [30/100], Loss: 0.3341
Epoch [40/100], Loss: 0.4599
Epoch [50/100], Loss: 0.4366
Epoch [60/100], Loss: 0.3822
Epoch [70/100], Loss: 0.3261
Epoch [80/100], Loss: 0.4239
Epoch [90/100], Loss: 0.2987
Epoch [100/100], Loss: 0.3993


## Fit Model2

In [15]:
criterion = F.mse_loss
optimizer = torch.optim.SGD(model2.parameters(), lr=1e-4)
fit(100, model1, criterion, optimizer, train_loader)

Epoch [10/100], Loss: 0.3251
Epoch [20/100], Loss: 0.3310
Epoch [30/100], Loss: 0.3169
Epoch [40/100], Loss: 0.4198
Epoch [50/100], Loss: 0.3816
Epoch [60/100], Loss: 0.3746
Epoch [70/100], Loss: 0.3555
Epoch [80/100], Loss: 0.3297
Epoch [90/100], Loss: 0.4220
Epoch [100/100], Loss: 0.3926


## Fit Model2 with Regularization

In [17]:
criterion = F.mse_loss
optimizer = torch.optim.SGD(model2.parameters(),weight_decay=2 ,lr=1e-4)
fit(100, model1, criterion, optimizer, train_loader)

Epoch [10/100], Loss: 0.2925
Epoch [20/100], Loss: 0.3272
Epoch [30/100], Loss: 0.4146
Epoch [40/100], Loss: 0.3322
Epoch [50/100], Loss: 0.3497
Epoch [60/100], Loss: 0.3691
Epoch [70/100], Loss: 0.2905
Epoch [80/100], Loss: 0.4107
Epoch [90/100], Loss: 0.3415
Epoch [100/100], Loss: 0.3779
